## OUTLET DEFINITIONS ##

In [ ]:
## This script calculates the outlet positions as vector points, useful if you need the discharge constrained by other geographical factors (e.g. on fjord-scale)
## should be included as part of your main script, with the grass dependencies, mapset etc. already setup.
## as input you need: streams and flow_dir. optionally, add elevation and accumulation map. 
## You ned the add on 'r.stream.order', which you can get by running:

#gs.run_command('g.extension', extension = 'r.stream.order')

import geopandas as gpd

#----------Define stream orders to get the outlet streams (where next_stream=-1)----------# 
gs.run_command('r.stream.order', 
               stream_rast='streams', 
               direction='flow_dir', 
               elevation='dem_filled', 
               accumulation ='accum', 
               strahler='strahler', 
               stream_vect='streams_vect',
               overwrite=True) 

#----------Extract only streams with no next_stream ----------# 
gs.run_command('v.extract', 
               input='streams_vect', 
               output='outlet_segment', 
               where="next_stream = -1", 
               overwrite=True) 
#----------Get the end points of those streams ----------# 
gs.run_command('v.to.points', 
               input='outlet_segment', 
               output='outlet_point', 
               use='end', # downstream end 
               flags='r', 
               overwrite=True)
#----------export to gpkg to work w geopandas ----------# 
gs.run_command('v.out.ogr', input ='outlet_point', format ='GPKG', output = 'outlets.gpkg', overwrite=True)

# 1. Read the GeoPackage
gdf = gpd.read_file("outlets.gpkg")

# 2. Add a new column with running numbers starting at 1
gdf['new_id'] = range(1, len(gdf) + 1)

# 2. Keep only one point per stream (lowest id is apparently always the start of the stream, and the highest is the end of each stream)
gdf_clean = gdf.loc[gdf.groupby('stream')['new_id'].idxmax()]

# 3. Save cleaned outlets back to a new GeoPackage
gdf_clean.to_file(fr"{OUT}\outlets_clean.gpkg", driver="GPKG")

gs.run_command('v.import', input='outlets_clean.gpkg', output='outlet_point_clean', overwrite=True)

# Now you can do basin delineation based on the outlets. If you don't need the outlets, use the streams raster as input instead. It should create similar outputs
# read the r.stream.basins documentation for more information